In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import os
import re
import cv2
import pydicom
import nibabel as nib
from sklearn.preprocessing import LabelEncoder
from glob import glob
from sklearn import preprocessing
from keras.utils import to_categorical
from tensorflow.keras.models import load_model
import tensorflow as tf
from keras import backend as K
import math
tf.config.run_functions_eagerly(True)

In [ ]:
def dice_coef(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    smooth = 0.0001
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_multilabel(y_true, y_pred, numLabels = 3):
    dice = 0
    weights = [0.348897, 16.0187006, 14.00607425]
    for index in range(numLabels):
        dice += dice_coef(y_true[:,:,index], y_pred[:,:,index]) * weights[index]
    return dice/np.sum(weights)

def dice_coef_multilabelloss(y_true, y_pred):
    return 1 - dice_coef_multilabel(y_true, y_pred)

def weightedLoss(originalLossFunc, weightsList):

    def lossFunc(true, pred):
        true = K.cast(true, K.floatx())
        pred = K.cast(pred, K.floatx())

        axis = -1 #if channels last
          #axis=  1 #if channels first


          #argmax returns the index of the element with the greatest value
          #done in the class axis, it returns the class index
        classSelectors = K.argmax(true, axis=axis)
              #if your loss is sparse, use only true as classSelectors

          #considering weights are ordered by class, for each class
          #true(1) if the class index is equal to the weight index
          #weightsList = tf.cast(weightsList, tf.int64)
        classSelectors = [K.equal(tf.cast(i, tf.int64), tf.cast(classSelectors, tf.int64)) for i in range(len(weightsList))]

          #casting boolean to float for calculations
          #each tensor in the list contains 1 where ground true class is equal to its index
          #if you sum all these, you will get a tensor full of ones.
        classSelectors = [K.cast(x, K.floatx()) for x in classSelectors]

          #for each of the selections above, multiply their respective weight
        weights = [sel * w for sel,w in zip(classSelectors, weightsList)]

          #sums all the selections
          #result is a tensor with the respective weight for each element in predictions
        weightMultiplier = weights[0]
        for i in range(1, len(weights)):
            weightMultiplier = weightMultiplier + weights[i]


          #make sure your originalLossFunc only collapses the class axis
          #you need the other axes intact to multiply the weights tensor
        loss = originalLossFunc(true,pred)
        loss = loss * weightMultiplier

        return loss
    return lossFunc

### Data Generation

In [ ]:
metadata_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv"

train_metadata = pd.read_csv(metadata_path)
segmentations_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations"

segmentations = os.listdir(segmentations_path)
segmentations = [int(os.path.splitext(segmentation)[0]) for segmentation in segmentations]


series = train_metadata["series_id"].tolist()

matched_series = []

for segmentation in segmentations:
    if segmentation in series:
        matched_series.append(segmentation)
    else:
        continue

patients_segment = train_metadata[train_metadata["series_id"].isin(matched_series)].reset_index(drop=True)
patients_with_segmentations = patients_segment["patient_id"].unique()


In [ ]:
def extract_number_from_path(path):
    match = re.search(r'(\d+)\.dcm$', path)
    if match:
        return int(match.group(1))
    return 0

def get_data_for_3d_volumes(data, dcm_path, niftii_path):

    data_to_merge = data[["patient_id", "series_id"]]
    
    total_paths = []
    patient_ids = []
    series_ids = []
    seg_path = []
    
    for patient_id in range(len(data_to_merge)):
    
        p_id = str(data_to_merge["patient_id"][patient_id]) + "/" + str(data_to_merge["series_id"][patient_id])
        str_imgs_path = dcm_path + p_id + '/'
        
        seg_mask_paths = niftii_path + str(data_to_merge["series_id"][patient_id]) + ".nii"
        seg_path.append(seg_mask_paths)
        
        patient_img_paths = []

        for file in glob(str_imgs_path + '/*'):
            patient_img_paths.append(file)
        
        
        sorted_file_paths = sorted(patient_img_paths, key=extract_number_from_path)
        total_paths.append(sorted_file_paths)
        patient_ids.append(data_to_merge["patient_id"][patient_id])
        series_ids.append(data_to_merge["series_id"][patient_id])
    
    final_data = pd.DataFrame(list(zip(patient_ids, series_ids, total_paths, seg_path)),
               columns =["patient_id","series_id", "patient_paths", "patient_segmentation"])
    
    return final_data

In [ ]:
dcm_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/"
niftii_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations/"

cleaned_data = get_data_for_3d_volumes(patients_segment, dcm_path, niftii_path)
cleaned_data.head(10)

In [ ]:
def window_converter(image, window_width=400, window_level=50):

    img_min = window_level - window_width // 2
    img_max = window_level + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    #image = (image / image.max() * 255).astype(np.float64)
    return window_image

def transform_to_hu(medical_image, image):
    meta_image = pydicom.dcmread(medical_image)
    intercept = meta_image.RescaleIntercept
    slope = meta_image.RescaleSlope
    hu_image = image * slope + intercept
    return hu_image

def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    # Correct DICOM pixel_array if PixelRepresentation == 1.
        pixel_array = dcm.pixel_array
        if dcm.PixelRepresentation == 1:
            bit_shift = dcm.BitsAllocated - dcm.BitsStored
            dtype = pixel_array.dtype 
            pixel_array = (pixel_array << bit_shift).astype(dtype) >> bit_shift
        return pixel_array

def resize_img(img_paths, target_size=(128, 128)):
        volume_shape = (target_size[0], target_size[1], len(img_paths)) 
        volume = np.zeros(volume_shape, dtype=np.float64)
        for i, image_path in enumerate(img_paths):
            image = pydicom.read_file(image_path)
            image = standardize_pixel_array(image)
            hu_image = transform_to_hu(image_path, image)
            window_image = window_converter(hu_image)
            image = cv2.resize(window_image, target_size)
            volume[:,:,i] = image
        return volume
    
def normalize_volume(resized_volume):
    original_shape = resized_volume.shape
    flattened_image = resized_volume.reshape((-1,))
    scaler = preprocessing.MinMaxScaler()
    normalized_flattened_image = scaler.fit_transform(flattened_image.reshape((-1, 1)))
    normalized_volume_image = normalized_flattened_image.reshape(original_shape)
    return normalized_volume_image

def create_3D_segmentations(filepath, target_size, downsample_rate=1):
    img = nib.load(filepath).get_fdata()
    img = np.transpose(img, [2, 1, 0])
    img = np.rot90(img, -1, (1,2))
    img = img[::-1,:,:]
    img = np.transpose(img, [2, 1, 0])
    img = img[::downsample_rate, ::downsample_rate, ::downsample_rate]
    
    resized_images = []

    for i in range(img.shape[2]):
        resized_img = cv2.resize(img[:, :, i], target_size)
        resized_images.append(resized_img)
    
    resized_3D_mask = np.stack(resized_images, axis=2)
    
    return np.array(resized_3D_mask, dtype=np.int8)

def generate_patient_processed_data(list_img_paths, list_seg_paths, target_size=(128,128)):

    height = target_size[0]
    width = target_size[1]
    depth = len(list_img_paths)

    volume_array = np.zeros((height, width, depth), dtype=np.float64)

    print("Initializing data preprocessing with the following dimensions-> Volumes:{}".format(volume_array.shape))

    resized_images = resize_img(list_img_paths, target_size=target_size)
    normalized_siz_volume = normalize_volume(resized_images)
    volume_array = normalized_siz_volume
    volume_mask = create_3D_segmentations(list_seg_paths, target_size=target_size)

    return volume_array, volume_mask

def compute_class_weights_and_encode_masks(volume_segmentations):
    
    volume_segmentations = np.transpose(volume_segmentations, (2, 0, 1))
    # Extract unique values from labels 0, 1, and 5
    labels_to_extract = [0, 1, 5]
    filtered_mask = np.isin(volume_segmentations, labels_to_extract)

    # Create a mask with zeros initially
    result_mask = np.zeros_like(volume_segmentations)

    # Replace the values in the result mask with the values from the filtered labels
    for label in labels_to_extract:
        result_mask[volume_segmentations == label] = label
    
    labelencoder = LabelEncoder()
    n, h, w = result_mask.shape
    train_masks_reshaped = result_mask.reshape(-1,1)
    train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped.ravel())
    train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)
    number_classes = len(np.unique(train_masks_reshaped_encoded))

    return train_masks_encoded_original_shape, number_classes

def transpose_and_expand_data(volume_images, volume_masks_encoded):
    transposed_volume_dcm = np.transpose(volume_images, (2, 0, 1))
    transposed_volume_dcm = np.expand_dims(transposed_volume_dcm, axis=3)
    transpose_volume_nii = np.expand_dims(volume_masks_encoded, axis=3)
    print(f"Final data shape: {transposed_volume_dcm.shape}, {transpose_volume_nii.shape}")
    return transposed_volume_dcm, transpose_volume_nii

def generate_data_volumes(data, idx):
    volume_dcm = []
    volume_nii = []
    for i in range(idx):
        volume_img, volume_seg = generate_patient_processed_data(data["patient_paths"][i], data["patient_segmentation"][i])
        volume_dcm.append(volume_img)
        volume_nii.append(volume_seg)
    volume_of_imgs = np.concatenate(volume_dcm, axis=2)
    volume_of_segs = np.concatenate(volume_nii, axis=2)
    return volume_of_imgs, volume_of_segs

def string_to_list(string_repr):
    return eval(string_repr)

### Inference using U-Net model

In [ ]:
volume_of_imgs, volume_of_segs = generate_data_volumes(data=cleaned_data, idx=1)

encoded_masks, num_classes = compute_class_weights_and_encode_masks(volume_of_segs)

volume_images_cleaned, volume_segs_cleaned = transpose_and_expand_data(volume_images=volume_of_imgs, volume_masks_encoded=encoded_masks)

In [ ]:
print(volume_images_cleaned.shape, volume_segs_cleaned.shape)

In [ ]:
train_masks_cat = to_categorical(volume_segs_cleaned, num_classes=6)
y_train_cat = train_masks_cat.reshape((volume_segs_cleaned.shape[0], volume_segs_cleaned.shape[1], volume_segs_cleaned.shape[2], 6))

### Loading model

In [ ]:
model = load_model('/kaggle/input/unet-model/UnetV_128_epochs.h5', custom_objects={'lossFunc': weightedLoss, 'dice_coef_multilabel': dice_coef_multilabel})

In [ ]:
class_weights = [0.17649986,  7.78453571, 41.53978194, 65.20657672, 96.75504125,  6.40743063]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=weightedLoss(tf.keras.losses.categorical_crossentropy, class_weights), metrics=[dice_coef_multilabel, tf.keras.metrics.MeanIoU(num_classes=6), "accuracy"])

In [ ]:
evaluation = model.evaluate(volume_images_cleaned, y_train_cat)

In [ ]:
y_pred = model.predict(volume_images_cleaned)
y_pred_argmax=np.argmax(y_pred, axis=3)

In [ ]:
plt.imshow(y_pred[300,:,:,1], cmap="jet")

In [ ]:
index = 300

fig, axes = plt.subplots(1, 3, figsize=(10, 4))

axes[0].imshow(volume_images_cleaned[index,:,:], cmap='gray')
axes[0].set_title('Slice')

axes[1].imshow(volume_segs_cleaned[index,:,:], cmap='jet')
axes[1].set_title('Ground truth')

axes[2].imshow(y_pred_argmax[index,:,:], cmap='jet')
axes[2].set_title('Prediction')

plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
num_slices = 5
index = 176
#
# Create a figure and axes for subplots
fig, axes = plt.subplots(num_slices, 3, figsize=(15, 4 * num_slices))

# Loop through the slices and plot each one
for i in range(num_slices):
    axes[i, 0].imshow(volume_images_cleaned[index + i, :, :], cmap='gray')
    axes[i, 0].set_title(f'Slice {index + i}')

    axes[i, 1].imshow(volume_segs_cleaned[index + i, :, :], cmap='jet')
    axes[i, 1].set_title('Ground truth')

    axes[i, 2].imshow(y_pred_argmax[index + i, :, :], cmap='jet')
    axes[i, 2].set_title('Prediction')

# Adjust layout and spacing
plt.tight_layout()

# Show the plot
plt.show()